In [1]:
import cv2
import numpy as np
import os
import face_recognition as fr
import csv
from datetime import datetime, date

In [2]:
path='Images'

In [3]:
os.listdir(path)

['Abhishek.jpeg',
 'Adhith.jpeg',
 'Adisha.jpeg',
 'Adithya.jpeg',
 'Aiswarya.jpeg',
 'Ashikh.jpeg',
 'Hridya.jpeg',
 'Jithin.jpeg',
 'Sanvi.jpeg',
 'Sreenand.jpeg',
 'Syam.jpeg']

In [4]:
mylist=os.listdir(path)
mylist

['Abhishek.jpeg',
 'Adhith.jpeg',
 'Adisha.jpeg',
 'Adithya.jpeg',
 'Aiswarya.jpeg',
 'Ashikh.jpeg',
 'Hridya.jpeg',
 'Jithin.jpeg',
 'Sanvi.jpeg',
 'Sreenand.jpeg',
 'Syam.jpeg']

In [5]:
imgs=[]         
classnames=[]   
for i in mylist:
    imgpath=os.path.join(path,i)
    current_img=cv2.imread(imgpath)

    imgs.append(current_img)

    classnames.append(i.split(".")[0])

In [6]:
classnames

['Abhishek',
 'Adhith',
 'Adisha',
 'Adithya',
 'Aiswarya',
 'Ashikh',
 'Hridya',
 'Jithin',
 'Sanvi',
 'Sreenand',
 'Syam']

In [7]:
def face_encodings(images):
    encodelist=[]
    for img in imgs:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        face_in_frame=fr.face_locations(img)
      
        face_encoder=fr.face_encodings(img,face_in_frame)[0]
        
        encodelist.append(face_encoder)
    return encodelist

In [8]:
encodelist_knownfaces=face_encodings(imgs)
encodelist_knownfaces

[array([-0.21588182,  0.05076535,  0.03780815, -0.02051405,  0.01545362,
        -0.06442932,  0.02175345, -0.03925498,  0.18872027, -0.10848811,
         0.17462029,  0.00996671, -0.15939368, -0.11530332, -0.07846275,
         0.10299031, -0.13127154, -0.15359905, -0.03955025, -0.09105109,
         0.06188465,  0.03723881, -0.03483754,  0.06516887, -0.14523777,
        -0.34937558, -0.09007461, -0.13727608,  0.03451394, -0.07341261,
         0.02438628,  0.00456741, -0.20870233, -0.04999656, -0.03091788,
         0.06353078,  0.09432939,  0.03955122,  0.16239695, -0.00216546,
        -0.12166528, -0.05164617,  0.02865955,  0.30675519,  0.11524586,
         0.06163082,  0.00557964, -0.02063326,  0.09395368, -0.2639387 ,
         0.07363991,  0.09172045,  0.09684914, -0.00166158,  0.08497006,
        -0.09276029,  0.0035943 ,  0.07083244, -0.25130036,  0.11581482,
         0.01877785, -0.04876288, -0.09288655,  0.0112788 ,  0.21657582,
         0.13182756, -0.1007674 , -0.14609081,  0.2

In [9]:
attendance_file = "attendance.csv"

if not os.path.exists(attendance_file):
    with open(attendance_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Name", "Date", "Time"])

In [10]:
attendance_list = []

In [11]:
video=cv2.VideoCapture(0)   
while True:   
    suc,frame=video.read()
    frame1=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    face_in_frame=fr.face_locations(frame1)
   
    encode_face=fr.face_encodings(frame1,face_in_frame)
  
    for enc_face,faceloc in zip(encode_face,face_in_frame):
        matches=fr.compare_faces(encodelist_knownfaces,enc_face)
        
        face_dis=fr.face_distance(encodelist_knownfaces,enc_face)
       
        matchindex=np.argmin(face_dis)
       
        if matches[matchindex]:
          name = classnames[matchindex].upper()

          today = date.today().isoformat()
          time_now = datetime.now().strftime("%H:%M:%S")

          #To read all existing attendance file
          with open(attendance_file, 'r') as f:
               lines = f.readlines()

          #Prevent duplicate
          already_marked = False
          for line in lines:
               if name in line and today in line:
                    already_marked = True
                    break

          if not already_marked:
               with open(attendance_file, 'a', newline='') as f:
                    writer = csv.writer(f)
                    writer.writerow([name, today, time_now])

        else:
             name="Unknown"
             print(name)
        y1,x2,y2,x1=faceloc
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),3)
        cv2.putText(frame,name,(x1,y1),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)



    cv2.imshow('Face',frame)
    if cv2.waitKey(1) & 0XFF==ord('q'): 
            break
video.release()
cv2.destroyAllWindows()

Unknown
